# 멜론 음원 차트 크롤링

## 연도별

In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

# 웹 드라이버 실행
driver1 = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')
driver2 = webdriver.Chrome('./chromedriver_win32/chromedriver.exe')

url = "https://www.melon.com/chart/index.htm"
driver1.get(url)

song_data = []
year = 2020

# 차트 파인더 조건 선택
driver1.find_elements_by_css_selector('li.nth1.on > div > div > button')[0].click()  # 차트 파인더 페이지로 이동
time.sleep(1)
driver1.find_elements_by_css_selector('h4.tab03 > a')[0].click()  # 차트 선택
time.sleep(1)
for i in range(1,3):
    driver1.find_elements_by_css_selector(f'div.box_chic.nth1.view > div.list_value > ul > li:nth-child({i})')[0].click()  # 연대 선택
    time.sleep(1)
    for j in range(1,4):
        try:
            driver1.find_elements_by_css_selector(f'div.box_chic.nth2.view > div.list_value > ul > li:nth-child({j})')[0].click()  # 연도 선택
            time.sleep(1)
        except IndexError :
            continue
        driver1.find_elements_by_css_selector('div.box_chic.last.view > div.list_value > ul > li:nth-child(2)')[0].click()  # 장르/스타일 선택
        time.sleep(1)
        driver1.find_elements_by_css_selector('div.wrap_btn_serch > button')[0].click()  # 검색 버튼 클릭
        time.sleep(1)

        html = driver1.page_source
        soup = BeautifulSoup(html, 'html.parser')
        songs = soup.select('table > tbody > tr')
        rank = 1

        for song in songs:
            title = song.select('div.rank01 > span > strong > a')[0].text  # 제목 추출
            singer = song.select('div.rank02 > a')[0].text  # 가수 추출

            href = song.select('a.btn_icon_detail')[0].get('href')  # 곡 정보 버튼에서 href 속성값 가져오기
            songId = re.findall("\d+", href)[0]  # songId 추출

            url = f'https://www.melon.com/song/detail.htm?songId={songId}'  # 곡 정보 페이지 이동
            driver2.get(url)

            genre = driver2.find_elements_by_css_selector('div.meta > dl > dd:nth-child(6)')[0].text  # 장르 추출

            song_data.append([year, rank, title, singer, genre])  # 추출한 데이터 리스트로 저장
            rank += 1
            time.sleep(3)
        year -= 1


# 데이터 프레임으로 만든 후 엑셀파일로 저장
columns = ['년도', '순위', '타이틀', '가수', '장르']
pd_data = pd.DataFrame(song_data, columns = columns)
pd_data.to_excel('./data/melon_year.xlsx', index=False)